# 📁 File Operations Using Code Interpreter

Welcome to the File Operations module using AWS Bedrock AgentCore Code Interpreter! This notebook demonstrates comprehensive file operations including reading, writing, and processing files using both boto3 and AgentCore SDK.

## 🎯 Learning Objectives

In this notebook, you'll learn how to:

- 🔧 Set up and use AgentCore Code Interpreter for file operations
- 📄 Read and write files using Code Interpreter's file management tools
- 📊 Process JSON and CSV files with both JavaScript and Python
- 🔄 Use both boto3 and AgentCore SDK for file I/O operations
- 🏗️ Implement complete file workflows including upload, process, and verify

## 📋 Prerequisites

- AWS account with access to Amazon Bedrock AgentCore
- Proper IAM permissions for AgentCore services
- Python 3.12 environment with required dependencies

Let's begin! 🚀

## Amazon Bedrock AgentCore Code Interpreter


The Amazon Bedrock AgentCore Code Interpreter enables AI agents to write and execute code securely in sandbox environments, enhancing their accuracy and expanding their ability to solve complex end-to-end tasks. This is critical in Agentic AI applications where the agents may execute arbitrary code that can lead to data compromise or security risks. The AgentCore Code Interpreter tool provides secure code execution, which helps you avoid running into these issues.

The Code Interpreter comes with pre-built runtimes for multiple languages and advanced features, including large file support and internet access, and CloudTrail logging capabilities. For inline upload, the file size can be up to 100 MB. And for uploading to Amazon S3 through terminal commands, the file size can be as large as 5 GB.

Developers can customize environments with session properties and network modes to meet their enterprise and security requirements. The AgentCore Code Interpreter reduces manual intervention while enabling sophisticated AI development without compromising security or performance.

![Amazon Bedrock AgentCore Code Interpreter Architecture](../../../media/agent_core_code_interpreter_arch.png)

## 🔧 Section 1: Environment Setup

In [ ]:
# 🔧 Environment Setup Instructions

# IMPORTANT: This notebook requires Python 3.12.8 as specified in pyproject.toml
# All dependencies are managed via UV (recommended approach for this project)

# SETUP STEPS:
# 1. Create virtual environment: uv venv
# 2. Install all dependencies: uv sync
# 3. Launch Jupyter Lab: uv run jupyter lab

# FOR VS CODE USERS:
# 1. Run: uv run python -c "import sys; print(sys.executable)"
# 2. Copy the path and select it as your Python interpreter in VS Code
# 3. The kernel should now have access to all required dependencies

# Verify environment setup
import sys
print(f"✅ Python version: {sys.version}")
print(f"✅ Python executable: {sys.executable}")
print("✅ Environment setup complete! All dependencies should be available via UV.")

In [ ]:
# Import core dependencies
import boto3
import json
import os
import time
from typing import Dict, Any, List, Optional

# Import AgentCore SDK
from bedrock_agentcore.tools.code_interpreter_client import CodeInterpreter, code_session

# Import existing Module 3 components
from agentic_platform.core.models.memory_models import Message, SessionContext
from agentic_platform.core.models.llm_models import LLMRequest, LLMResponse
from agentic_platform.core.models.api_models import AgenticRequest, AgenticResponse

print("✅ Dependencies imported successfully!")

In [ ]:
# Initialize AWS clients with specific profile
profile_name = "XXXXXX" # your aws profile name
session = boto3.Session(profile_name=profile_name)
region_name = "us-west-2"

# Verify credentials are working
try:
    credentials = session.get_credentials()
    print(f"✅ Using AWS profile: {profile_name}")
    print(f"✅ Access Key: {credentials.access_key[:10]}...")
    print(f"✅ Region: {region_name}")
except Exception as e:
    print(f"❌ Error with AWS credentials: {e}")
    raise

# Standard AgentCore client
bedrock_agentcore_client = session.client(
    'bedrock-agentcore',
    region_name=region_name,
    endpoint_url=f"https://bedrock-agentcore.{region_name}.amazonaws.com"
)

# Set environment variables for AgentCore SDK
os.environ['AWS_ACCESS_KEY_ID'] = credentials.access_key
os.environ['AWS_SECRET_ACCESS_KEY'] = credentials.secret_key
if credentials.token:
    os.environ['AWS_SESSION_TOKEN'] = credentials.token
os.environ['AWS_DEFAULT_REGION'] = region_name

print("✅ AWS clients initialized successfully with profile!")

## 📄 Section 2: Sample Data Preparation

Let's prepare sample data files that we'll use throughout this notebook.

In [ ]:
# Prepare sample CSV data
sample_csv_data = """name,age,city,occupation
Alice Johnson,28,New York,Software Engineer
Bob Smith,35,San Francisco,Data Scientist
Carol Williams,42,Chicago,Product Manager
David Brown,31,Austin,DevOps Engineer
Emma Davis,26,Seattle,UX Designer"""

# Prepare sample Python analysis script
stats_py_content = """import csv
from io import StringIO

# Read the CSV data
with open('data.csv', 'r') as file:
    csv_reader = csv.DictReader(file)
    data = list(csv_reader)

# Calculate statistics
total_people = len(data)
ages = [int(row['age']) for row in data]
avg_age = sum(ages) / len(ages)
min_age = min(ages)
max_age = max(ages)

# Count by city
cities = {}
for row in data:
    city = row['city']
    cities[city] = cities.get(city, 0) + 1

# Print results
print(f\"Dataset Statistics:\")
print(f\"==================\")
print(f\"Total people: {total_people}\")
print(f\"Average age: {avg_age:.1f}\")
print(f\"Age range: {min_age} - {max_age}\")
print(f\"\\nPeople by city:\")
for city, count in cities.items():
    print(f\"  {city}: {count}\")

# Create a summary report
report = {
    'total_records': total_people,
    'average_age': round(avg_age, 2),
    'age_range': {'min': min_age, 'max': max_age},
    'city_distribution': cities
}

print(f\"\\nSummary report generated successfully!\")
print(f\"Report: {report}\")
"""

# Prepare sample JSON data
sample_json_data = json.dumps({
    "project": "File Operations Demo",
    "version": "1.0.0",
    "features": ["read", "write", "process", "analyze"],
    "config": {
        "debug": False,
        "timeout": 300,
        "max_retries": 3
    }
}, indent=2)

print("✅ Sample data prepared:")
print(f"  - CSV data: {len(sample_csv_data)} bytes")
print(f"  - Python script: {len(stats_py_content)} bytes")
print(f"  - JSON data: {len(sample_json_data)} bytes")

## 🔧 Section 3: File Operations with Boto3

This section demonstrates file read/write operations using boto3 direct API calls following the pattern from the provided example.

In [ ]:
def call_tool_boto3(session_id: str, tool_name: str, arguments: Dict[str, Any]) -> Dict[str, Any]:
    """Helper function to call Code Interpreter tools using boto3"""
    response = bedrock_agentcore_client.invoke_code_interpreter(
        codeInterpreterIdentifier="aws.codeinterpreter.v1",
        sessionId=session_id,
        name=tool_name,
        arguments=arguments
    )
    
    for event in response["stream"]:
        if "result" in event:
            return event["result"]
    return {}

In [ ]:
# Start boto3 Code Interpreter session
print("🚀 Starting boto3 Code Interpreter session...")
session_response = bedrock_agentcore_client.start_code_interpreter_session(
    codeInterpreterIdentifier="aws.codeinterpreter.v1",
    name="boto3-file-ops-session",
    sessionTimeoutSeconds=300
)
boto3_session_id = session_response["sessionId"]
print(f"✅ Session started: {boto3_session_id}")

In [ ]:
# Prepare files to create
files_to_create = [
    {
        "path": "data.csv",
        "text": sample_csv_data
    },
    {
        "path": "stats.py",
        "text": stats_py_content
    },
    {
        "path": "config.json",
        "text": sample_json_data
    }
]

# Write files to the session
print("\n📝 Writing files to Code Interpreter session...")
writing_files = call_tool_boto3(boto3_session_id, "writeFiles", {"content": files_to_create})
print(f"✅ Files written: {json.dumps(writing_files, indent=2)}")

In [ ]:
# List files to verify
print("\n📋 Listing files in session...")
listing_files = call_tool_boto3(boto3_session_id, "listFiles", {"path": ""})
print(f"📁 Files in session: {json.dumps(listing_files, indent=2)}")

In [ ]:
# Execute the Python script to analyze the CSV data
print("\n🔧 Executing analysis script...")
execute_code = call_tool_boto3(boto3_session_id, "executeCode", {
    "code": stats_py_content,
    "language": "python",
    "clearContext": True
})
print(f"📊 Code execution result: {json.dumps(execute_code, indent=2)}")

In [ ]:
# Read a file back to verify
print("\n📖 Reading config.json to verify...")
read_file = call_tool_boto3(boto3_session_id, "readFiles", {"path": "config.json"})
print(f"📄 File content: {json.dumps(read_file, indent=2)}")

In [ ]:
# Clean up boto3 session
bedrock_agentcore_client.stop_code_interpreter_session(
    codeInterpreterIdentifier="aws.codeinterpreter.v1",
    sessionId=boto3_session_id
)
print(f"\n✅ Boto3 session {boto3_session_id} stopped")

## 🔧 Section 4: File Operations with AgentCore SDK

This section demonstrates the same file operations using the AgentCore SDK's higher-level interface.

In [ ]:
# Configure and start the code interpreter session
print("🚀 Starting AgentCore SDK Code Interpreter session...")
code_client = CodeInterpreter(region_name)
code_client.start()
print("✅ AgentCore SDK session started")

In [ ]:
# Define the method to call the invoke API (following the example pattern)
def call_tool(tool_name: str, arguments: Dict[str, Any]) -> Dict[str, Any]:
    """Helper function to call Code Interpreter tools using SDK"""
    response = code_client.invoke(tool_name, arguments)
    for event in response["stream"]:
        return json.dumps(event["result"], indent=2)

In [ ]:
# Write the sample data and analysis script into the code interpreter session
print("\n📝 Writing files using SDK...")
writing_files = call_tool("writeFiles", {"content": files_to_create})
print(f"✅ Files written: {writing_files}")

In [ ]:
# List and validate that the files were written successfully
print("\n📋 Listing files using SDK...")
listing_files = call_tool("listFiles", {"path": ""})
print(f"📁 Files in session: {listing_files}")

In [ ]:
# Run the python script to analyze the sample data file
print("\n🔧 Executing analysis script using SDK...")
execute_code = call_tool("executeCode", {
    "code": files_to_create[1]['text'],
    "language": "python",
    "clearContext": True
})
print(f"📊 Code execution result: {execute_code}")

In [ ]:
# Read a file back to verify
print("\n📖 Reading config.json using SDK...")
read_file = call_tool("readFiles", {"path": "config.json"})
print(f"📄 File content: {read_file}")

In [ ]:
# Clean up and stop the code interpreter session
code_client.stop()
print("\n✅ AgentCore SDK session stopped")

## Conclusion

In this notebook, we've successfully demonstrated:

1. **File Operations with Boto3**:
   - Direct API calls to Code Interpreter
   - Writing files using `writeFiles`
   - Listing files using `listFiles`
   - Reading files using `readFile`
   - Executing code that processes files

2. **File Operations with AgentCore SDK**:
   - Higher-level interface for Code Interpreter
   - Simplified session management
   - Same file operations with cleaner syntax

### 📚 Additional Resources

- [AWS Bedrock AgentCore Documentation](https://docs.aws.amazon.com/bedrock-agentcore/)
- [Code Interpreter API Reference](https://docs.aws.amazon.com/bedrock-agentcore/latest/APIReference/)
- [Module 3 Other Notebooks](../README.md)

Happy coding! 🎉